In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [ ]:
# Load NVIDIA stock data from Yahoo Finance
nvidia = yf.download('NVDA', start='2018-01-01', end='2024-01-01', interval='1d')
nvidia.head()


In [ ]:
# Define technical indicator functions
def moving_average(df, window):
    return df['Close'].rolling(window=window).mean()

def exponential_moving_average(df, window):
    return df['Close'].ewm(span=window, adjust=False).mean()

def momentum(df, window):
    return df['Close'].diff(window)

def rsi(df, window):
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# Calculate technical indicators
nvidia['SMA10'] = moving_average(nvidia, 10)
nvidia['SMA60'] = moving_average(nvidia, 60)
nvidia['EMA10'] = exponential_moving_average(nvidia, 10)
nvidia['EMA20'] = exponential_moving_average(nvidia, 20)
nvidia['EMA50'] = exponential_moving_average(nvidia, 50)
nvidia['Momentum'] = momentum(nvidia, 10)
nvidia['RSI'] = rsi(nvidia, 14)

# Drop NaN values
nvidia.dropna(inplace=True)
nvidia.head()


# Simple EMA 12 - EMA 26 crossover strategy

In [ ]:
# Define the short and long window periods
short_window = 12
long_window = 26

# Calculate the EMAs
nvidia['EMA12'] = nvidia['Close'].ewm(span=short_window, adjust=False).mean()
nvidia['EMA26'] = nvidia['Close'].ewm(span=long_window, adjust=False).mean()

# Initialize signals
nvidia['Signal'] = 0

# Split the data into training and testing sets
train_size = int(len(nvidia) * 0.8)
train_df = nvidia[:train_size]
test_df = nvidia[train_size:]

# Generate buy signals
nvidia.loc[nvidia.index[short_window:], 'Signal'] = np.where(nvidia['EMA12'][short_window:] > nvidia['EMA26'][short_window:], 1, 0)

# Generate trading orders
nvidia['Position'] = nvidia['Signal'].diff()

# Visualize signals
plt.figure(figsize=(14, 7))
plt.plot(nvidia['Close'], label='Close Price', alpha=0.5)
plt.plot(nvidia['EMA12'], label='EMA 12', alpha=0.75)
plt.plot(nvidia['EMA26'], label='EMA 26', alpha=0.75)
plt.plot(nvidia[nvidia['Position'] == 1].index, nvidia['EMA12'][nvidia['Position'] == 1], '^', markersize=10, color='g', label='Buy Signal')
plt.plot(nvidia[nvidia['Position'] == -1].index, nvidia['EMA12'][nvidia['Position'] == -1], 'v', markersize=10, color='r', label='Sell Signal')
plt.title('NVIDIA Stock Price with EMA Crossover Strategy')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


In [ ]:
# Function to calculate cumulative returns and performance metrics
def calculate_performance_metrics(df, signal_col):
    df['returns'] = df['Close'].pct_change()
    df['strategy_returns'] = df[signal_col].shift(1) * df['returns']
    cumulative_returns = (1 + df['strategy_returns']).cumprod() - 1

    sharpe_ratio = (df['strategy_returns'].mean() / df['strategy_returns'].std()) * np.sqrt(252)
    annualized_return = df['strategy_returns'].mean() * 252
    annualized_volatility = df['strategy_returns'].std() * np.sqrt(252)
    annualized_sharpe_ratio = annualized_return / annualized_volatility

    return cumulative_returns, sharpe_ratio, annualized_sharpe_ratio

# Calculate cumulative returns and performance metrics for EMA crossover strategy
ema_cumulative_returns, ema_sharpe_ratio, ema_annualized_sharpe_ratio = calculate_performance_metrics(nvidia, 'Signal')

# Print performance metrics for EMA crossover strategy
print(f"EMA Crossover Sharpe Ratio: {ema_sharpe_ratio:.2f}")
print(f"EMA Crossover Annualized Sharpe Ratio: {ema_annualized_sharpe_ratio:.2f}")

# Plot cumulative returns for EMA crossover strategy
plt.figure(figsize=(12, 6))
plt.plot(ema_cumulative_returns, label='EMA Crossover Cumulative Returns')
plt.title('Cumulative Returns of EMA Crossover Trading Strategy')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()


# Machine Learning Strategy using Technical Indicators as Features

In [ ]:
# Prepare signals for ML strategy
nvidia['ml_signal'] = nvidia['Signal']

# Split the data into training and testing sets
train_size = int(len(nvidia) * 0.8)
train_df = nvidia[:train_size]
test_df = nvidia[train_size:]

# Features and target variable
X_train = train_df[['SMA10', 'SMA60', 'EMA10', 'EMA20', 'EMA50', 'Momentum', 'RSI']]
y_train = train_df['ml_signal']
X_test = test_df[['SMA10', 'SMA60', 'EMA10', 'EMA20', 'EMA50', 'Momentum', 'RSI']]
y_test = test_df['ml_signal']


In [ ]:
# Train RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=25)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Train XGBClassifier
xgb_model = XGBClassifier(n_estimators=25, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)

# Add predictions to the test dataframe
test_df['rf_signal'] = rf_predictions
test_df['xgb_signal'] = xgb_predictions


In [ ]:
# Function to calculate returns and Sharpe ratio
def calculate_performance(df, signal_col):
    df['returns'] = df['Close'].pct_change()
    df['strategy_returns'] = df[signal_col].shift(1) * df['returns']
    cumulative_returns = df['strategy_returns'].cumsum()

    sharpe_ratio = (df['strategy_returns'].mean() / df['strategy_returns'].std()) * np.sqrt(252)

    annualized_return = df['strategy_returns'].mean() * 252
    annualized_volatility = df['strategy_returns'].std() * np.sqrt(252)
    annualized_sharpe_ratio = annualized_return / annualized_volatility

    return cumulative_returns, sharpe_ratio, annualized_sharpe_ratio

# Calculate performance for RandomForest strategy
rf_cumulative_returns, rf_sharpe_ratio, rf_annualized_sharpe_ratio = calculate_performance(test_df, 'rf_signal')

# Calculate performance for XGBoost strategy
xgb_cumulative_returns, xgb_sharpe_ratio, xgb_annualized_sharpe_ratio = calculate_performance(test_df, 'xgb_signal')

# Print performance metrics
print(f"RandomForest Sharpe Ratio: {rf_sharpe_ratio:.2f}")
print(f"RandomForest Annualized Sharpe Ratio: {rf_annualized_sharpe_ratio:.2f}")
print(f"XGBoost Sharpe Ratio: {xgb_sharpe_ratio:.2f}")
print(f"XGBoost Annualized Sharpe Ratio: {xgb_annualized_sharpe_ratio:.2f}")

# Plot cumulative returns for ML strategies
plt.figure(figsize=(12, 6))
plt.plot(rf_cumulative_returns, label='RandomForest Cumulative Returns')
plt.plot(xgb_cumulative_returns, label='XGBoost Cumulative Returns')
plt.title('Cumulative Returns of ML-based Trading Strategies')
plt.legend()
plt.show()


In [ ]:
# Calculate positions for RandomForest and XGBoost strategies
test_df['rf_position'] = test_df['rf_signal'].diff()
test_df['xgb_position'] = test_df['xgb_signal'].diff()

# Plot cumulative returns and buy/sell signals for RandomForest strategy
plt.figure(figsize=(14, 7))
plt.plot(test_df['Close'], label='Close Price', alpha=0.5)
plt.plot(test_df['Close'][test_df['rf_position'] == 1], '^', markersize=10, color='g', label='RandomForest Buy Signal')
plt.plot(test_df['Close'][test_df['rf_position'] == -1], 'v', markersize=10, color='r', label='RandomForest Sell Signal')
plt.title('NVIDIA Stock Price with RandomForest Strategy Buy/Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

# Plot cumulative returns and buy/sell signals for XGBoost strategy
plt.figure(figsize=(14, 7))
plt.plot(test_df['Close'], label='Close Price', alpha=0.5)
plt.plot(test_df['Close'][test_df['xgb_position'] == 1], '^', markersize=10, color='g', label='XGBoost Buy Signal')
plt.plot(test_df['Close'][test_df['xgb_position'] == -1], 'v', markersize=10, color='r', label='XGBoost Sell Signal')
plt.title('NVIDIA Stock Price with XGBoost Strategy Buy/Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()
